# Script for Creating SQL Database

In [1]:
import sqlite3
import pandas as pd

pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)  

## DB and Table Creation
From DnD excel file

In [2]:
# If building db from scratch, run this cell
import os
db_name = "lldm.db"
if os.path.exists(db_name):
    os.remove(db_name)

In [3]:
db_name = "lldm.db"
try: 
    db = sqlite3.connect(db_name) 
    print("Database lldm.db formed.") 
except: 
    print("Database lldm.db not formed.")

Database lldm.db formed.


In [4]:
dfs = pd.read_excel('DnD.xlsx', sheet_name=None)
for table, df in dfs.items():
    try:
        print(table.upper().replace(' ','_').strip())
        df.columns = [x.replace(' ','_') for x in df.columns.values]
        df.to_sql(table.upper().replace(' ','_').strip(), db)
    except:
        continue

CHARACTER_SHEET
INVENTORY
WORLD_ITEMS
SETTINGS
NPCS
TREASURES
MONSTERS
PLOT
LOGS


In [5]:
# run query and return pandas dataframe
def run_query(db, query):
    try:
        df = pd.read_sql_query(query, db)
        return df
    # not a select statement
    except TypeError:
        cursor = db.cursor()
        cursor.execute(query)
        return    

In [6]:
sql_query = """
SELECT name FROM sqlite_master  
WHERE type='table';
"""
cursor = db.cursor()
cursor.execute(sql_query)
print(cursor.fetchall())

[('CHARACTER_SHEET',), ('INVENTORY',), ('WORLD_ITEMS',), ('SETTINGS',), ('NPCS',), ('TREASURES',), ('MONSTERS',), ('PLOT',), ('LOGS',)]


In [8]:
# single script
query = '''
CREATE TABLE IF NOT EXISTS CAMPAIGN (
	Campaign_ID INTEGER NOT NULL,
   	Setting TEXT NOT NULL,
    Start_Time TEXT NOT NULL,
    Current_Turns INTEGER NOT NULL DEFAULT 0
);

--ALTER TABLE CHARACTER_SHEET
--RENAME COLUMN index TO Character_ID;
--
--ALTER TABLE CHARACTER_SHEET
--ADD COLUMN Campaign_ID INTEGER;
--
--ALTER TABLE INVENTORY
--RENAME COLUMN index TO Item_ID;
--
--ALTER TABLE WORLD_ITEMS
--RENAME COLUMN index TO Item_ID;
--
--ALTER TABLE SETTINGS
--RENAME COLUMN index TO Setting_ID;
--
--ALTER TABLE NPCS
--RENAME COLUMN index TO NPC_ID;
--
--ALTER TABLE TREASURES
--RENAME COLUMN index TO Treasure_ID;
--
--ALTER TABLE MONSTERS
--RENAME COLUMN index TO Monster_ID;
--
--ALTER TABLE PLOT
--RENAME COLUMN index TO Plot_ID;
--
--ALTER TABLE PLOT
--ADD COLUMN Campaign_ID INTEGER;
--
--ALTER TABLE LOGS
--RENAME COLUMN index TO Log_ID;
--
--ALTER TABLE LOGS
--ADD COLUMN Campaign_ID INTEGER;
--
--ALTER TABLE LOGS
--ADD COLUMN Character_ID INTEGER;

CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY (
    Campaign_ID INTEGER NOT NULL,
	Character_ID INTEGER NOT NULL,
   	Item_ID INTEGER NOT NULL,
	Total_Quantity FLOAT DEFAULT 0
);

CREATE TABLE IF NOT EXISTS CHARACTER_INVENTORY_HISTORY (
    Campaign_ID INTEGER NOT NULL,
	Character_ID INTEGER NOT NULL,
   	Item_ID INTEGER NOT NULL,
	Quantity FLOAT DEFAULT 0,
    Obtain_Time DATETIME DEFAULT CURRENT_TIMESTAMP
);

CREATE VIEW IF NOT EXISTS CHARACTER_INVENTORY_DETAILS 
AS
SELECT
    a.Campaign_ID, a.Character_ID, a.Item_ID, a.Total_Quantity,
    b.Weapon_Name, b.Weapon_Description
FROM CHARACTER_INVENTORY a
JOIN WORLD_ITEMS b ON a.Item_ID = b.Item_ID;

CREATE VIEW IF NOT EXISTS CHARACTER_INVENTORY_HISTORY_DETAILS 
AS
SELECT
    a.Campaign_ID, a.Character_ID, a.Item_ID, a.Quantity, a.Modify_Time,
    b.Weapon_Name, b.Weapon_Description
FROM CHARACTER_INVENTORY a
JOIN WORLD_ITEMS b ON a.Item_ID = b.Item_ID;
'''

cursor = db.cursor()
cursor.executescript(query)

In [10]:
query = '''
SELECT * FROM CHARACTER_INVENTORY
'''
run_query(db, query)

,Campaign_ID,Character_ID,Item_ID,Quantity


In [39]:
db.close()